In [1]:
using Revise
using LinearAlgebra
using AdaptiveTransportMap
using Statistics
using Distributions
using Test
using ForwardDiff
using SpecialFunctions
using OrthoMatchingPursuit
using BenchmarkTools

┌ Info: Precompiling AdaptiveTransportMap [bdf749b0-1400-4207-80d3-e689c0e3f03d]
└ @ Base loading.jl:1278
┌ Warning: Type annotations on keyword arguments not currently supported in recipes. Type information has been discarded
└ @ RecipesBase ~/.julia/packages/RecipesBase/92zOw/src/RecipesBase.jl:116
┌ Warning: Type annotations on keyword arguments not currently supported in recipes. Type information has been discarded
└ @ RecipesBase ~/.julia/packages/RecipesBase/92zOw/src/RecipesBase.jl:116


In [207]:
Nx = 5
Ne = 100

X = randn(Nx, Ne) .* randn(Nx, Ne)

λ = 0.0
δ = 1e-8
γ = 2.0

2.0

In [210]:
# for k=1:Nx-1
Cgreedy = greedyfit(Nx, 2, deepcopy(X), 1, λ, δ, γ)
#     C = SparseRadialMapComponent(Nx, Cgreedy.p)
#     center_std!(C, sort(X; dims = 2); γ = γ)
#     x_opt = optimize(C, X, λ, δ)
#     modify_a!(C, x_opt)
#     @test norm(Cgreedy.a - C.a)<1e-10
# end

5-th component of a Sparse KR rearrangement of order p = [-1, 2, -1, -1, 2]


In [211]:
p = 2
order = p*ones(Int64, Nx)
order[1] = p
order[end] = p

order = [p; p; -1; p; p]

C = SparseRadialMapComponent(Nx, Cgreedy.p)
@time center_std!(C, sort(deepcopy(X); dims = 2); γ = γ)

  0.000050 seconds (36 allocations: 22.906 KiB)


In [212]:
@time x_opt = optimize(C, X, λ, δ)
modify_a!(C, x_opt)

  0.000226 seconds (2.58 k allocations: 359.984 KiB)


In [213]:
vcat(Cgreedy.a...)

8-element Array{Float64,1}:
 -0.23668117193971738
  8.1084370661199
 -7.075019920425056
 -1.818023454358032
  0.3459321460941695
  2.796520581639266
  1.9035729852795827e-9
  0.36367824846981966

In [204]:
x_opt

5-element Array{Float64,1}:
 -1.2964886010656647
  0.6352741592915871
  1.3246779509702715
  1.27894321780262
  0.2898800401084731

In [26]:
ψ_off, ψ_diag, dψ_diag = compute_weights(C, X)

([-0.05378078416194535 0.49952203640785126 … 0.4593093104130878 0.5228655427813467; -0.011333947793594104 0.4876823299884622 … 0.015434938646567392 0.0019252579806903407; … ; -0.3247045134153518 0.5387708943673685 … 0.4817019427879171 0.49578475186297244; 0.4943325628595328 0.28457879872466607 … 0.4717848897672617 0.5133337107957159], [-0.22850013876026065 0.28835864029462954 … 0.21876030833320614 0.08210738213419216; -0.10104950784442333 0.599563692022141 … 0.3849233020729854 0.15239314777530744; … ; -0.18199234841955458 0.37735661511679863 … 0.26445499165600983 0.09990818708710597; -0.4363873586395267 0.08396918211365156 … 0.10269778144589148 0.040279453614281424], [0.450588475396419 0.7543014944751246 … 0.3893005656530648 0.1476976722189226; 0.25148700852584277 0.8538632833066603 … 0.504183222356279 0.2401278803282731; … ; 0.38604536439181364 0.8394706937810503 … 0.43159916526853775 0.17281182497102315; 0.6658134605990345 0.34068304464183286 … 0.23602701323620484 0.08209140484516159

In [56]:
n_off = size(ψ_off, 2)
F = qr(ψ_off);

In [57]:
@time Q1 = Matrix(F.Q)[1:Ne,:];
size(Q1)

  0.000059 seconds (10 allocations: 29.906 KiB)


(100, 12)

In [58]:
P1 = Matrix(1.0*I, Ne, n_off);

In [59]:
@time F.Q*P1

  0.000046 seconds (4 allocations: 10.781 KiB)


100×12 Array{Float64,2}:
 -0.00506335  -0.122255    0.00665846  …  -0.104569     0.0155983
 -0.00106707  -0.119642    0.0211632      -0.153965     0.163455
 -0.0400545   -0.128521   -0.103929        0.0307356    0.0967685
  0.0350319   -0.08563     0.129333        0.00497985  -0.0923161
  0.00817666  -0.112442    0.0539373       0.144438     0.151932
  0.44248     -0.0325423  -0.230145    …   0.163701    -0.0481845
  0.123105    -0.0200941   0.053733       -0.0552943    0.160027
 -0.00187378  -0.120196    0.0182414      -0.0561715    0.0821768
  0.0304014   -0.0905935   0.119396       -0.0297554    0.0749438
  0.132774    -0.0177914   0.0308263       0.156252    -0.0185383
  0.0345666   -0.0861308   0.128404    …   0.00693815  -0.128395
 -0.00172968  -0.120098    0.0187637      -0.105744    -0.126566
 -0.0561024   -0.12063    -0.131025        0.0672764   -0.0296273
  ⋮                                    ⋱   ⋮           
 -0.0345539   -0.129684   -0.0903687      -0.0884659    0.0524598


In [60]:
[F.Q*ψ_diag

LoadError: [91msyntax: incomplete: premature end of input[39m

In [74]:
(P1*P1')

100×100 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.

In [71]:
@time ((P1*P1')*F.Q'*ψ_diag)[13,:]

  0.000491 seconds (12 allocations: 170.219 KiB)


5-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0

In [82]:
F.Q*[(F.Q'*ψ_diag)[1:n_off, axes(ψ_diag,2)]; zeros(Ne-n_off, size(ψ_diag,2))]

100×5 Array{Float64,2}:
 -0.320456   0.560536  0.489216  0.423874   0.264696
  0.154248   0.2591    0.179849  0.160165  -0.021255
 -0.0203326  0.683253  0.608326  0.508856   0.447219
 -0.268992   0.592457  0.543131  0.432407   0.151748
 -0.122952   0.56465   0.518281  0.40212    0.17713
 -1.05279    0.401822  0.381176  0.356636   0.44541
 -0.270367   0.662803  0.608551  0.541607   0.393149
 -0.21563    0.576003  0.506565  0.429778   0.228464
 -0.201049   0.560582  0.49742   0.414979   0.195981
 -0.269881   0.716923  0.658919  0.574763   0.384386
 -0.256367   0.611235  0.560839  0.423568   0.179988
 -0.476065   0.393635  0.358131  0.267109   0.0651158
 -0.334518   0.492344  0.443949  0.355204   0.193277
  ⋮                                        
 -0.21911    0.641268  0.566849  0.487391   0.292654
 -0.268899   0.699861  0.630139  0.528839   0.293027
 -0.153724   0.66199   0.601092  0.506374   0.270359
 -0.140452   0.584902  0.507534  0.398443   0.184839
 -0.287145   0.533479  0.483789 

In [66]:
@time Q1*Q1'*ψ_diag

  0.000073 seconds (4 allocations: 82.281 KiB)


100×5 Array{Float64,2}:
 -0.320456   0.560536  0.489216  0.423874   0.264696
  0.154248   0.2591    0.179849  0.160165  -0.021255
 -0.0203326  0.683253  0.608326  0.508856   0.447219
 -0.268992   0.592457  0.543131  0.432407   0.151748
 -0.122952   0.56465   0.518281  0.40212    0.17713
 -1.05279    0.401822  0.381176  0.356636   0.44541
 -0.270367   0.662803  0.608551  0.541607   0.393149
 -0.21563    0.576003  0.506565  0.429778   0.228464
 -0.201049   0.560582  0.49742   0.414979   0.195981
 -0.269881   0.716923  0.658919  0.574763   0.384386
 -0.256367   0.611235  0.560839  0.423568   0.179988
 -0.476065   0.393635  0.358131  0.267109   0.0651158
 -0.334518   0.492344  0.443949  0.355204   0.193277
  ⋮                                        
 -0.21911    0.641268  0.566849  0.487391   0.292654
 -0.268899   0.699861  0.630139  0.528839   0.293027
 -0.153724   0.66199   0.601092  0.506374   0.270359
 -0.140452   0.584902  0.507534  0.398443   0.184839
 -0.287145   0.533479  0.483789 

In [159]:
@testset "Verify updateqrfactUnblocked! for a vector" begin

    ψ = randn(100, 20)
    ϕ = randn(100)

    F = qrfactUnblocked(ψ)
    Fold = deepcopy(F)
    G = qrfactUnblocked(hcat(ψ, ϕ))
    F = updateqrfactUnblocked!(F, ϕ)
    @test norm(F.τ-G.τ)<1e-14
    @test norm(F.factors - G.factors)<1e-14
end

@testset "Verify updateqrfactUnblocked! for a matrix" begin

    ψ = randn(100, 20)
    ϕ = randn(100, 5)

    F = qrfactUnblocked(ψ)
    Fold = deepcopy(F)
    G = qrfactUnblocked(hcat(ψ, ϕ))
    F = updateqrfactUnblocked!(F, ϕ)
    @test norm(F.τ-G.τ)<1e-14
    @test norm(F.factors - G.factors)<1e-14
end

Test Summary:                              | Pass  Total
Verify updateqrfactUnblocked! for a vector |    2      2
Test Summary:                              | Pass  Total
Verify updateqrfactUnblocked! for a matrix |    2      2


Test.DefaultTestSet("Verify updateqrfactUnblocked! for a matrix", Any[], 2, false)